In [ ]:
"""
05 MARCH 2024
Adrien Corniere

Making boxplot from different mlr models
"""

import os
from percephone.core.recording import RecordingAmplDet
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rnd
import percephone.core.recording as pc
import percephone.analysis.mlr_models as mlrm
from percephone.plts.stats import boxplot, paired_boxplot
from multiprocessing import pool, cpu_count
%matplotlib inline

In [ ]:
directory = "/datas/Théo/Projects/Percephone/data/Amplitude_Detection/loop_format_tau_02/"
roi_info = directory + "FmKO_ROIs&inhibitory.xlsx"
directory = "C:\\Users\\acorniere\\Desktop\\percephone_data\\"
roi_info = pd.read_excel(directory + "FmKO_ROIs&inhibitory.xlsx")
folders = os.listdir(directory)
files = os.listdir(directory)
files_ = [file for file in files if file.endswith("synchro")]

In [ ]:
# def oprening_rec(fil,i):
#     rec = pc.RecordingAmplDet(directory + fil + "/", 0, fil, roi_info)
#     mlr_model, name_model = mlrm.r2_model(rec)
#     rec.mlr(mlr_model, name_model)
#     return rec
# 
# workers = cpu_count()
# pool = pool.ThreadPool(processes=workers)
# async_results = [pool.apply_async(oprening_rec, args=(file,i)) for i,file in enumerate(files_)]
# recs = [ar.get() for ar in async_results]

In [ ]:
df_exc = pd.DataFrame()
df_inh = pd.DataFrame()
genotypes = ["Genotype"]
tot_exc = ["Total n exc"]
tot_inh = ["Total n inh"]
perc_exc = ["% neurons significant"]
perc_inh = ["% neurons significant"]

# for rec in recs :
for folder in files_:
    path = directory + folder +"/"
    if os.path.isdir(path):
        print(folder)
        rec = pc.RecordingAmplDet(path, 0, roi_info, correction=False)
        mlr_model, name_model = mlrm.precise_stim_model(rec)
        rec.mlr(mlr_model, name_model)
        genotypes.append(rec.genotype)
        df_exc["labels"] = rec.mlr_labels_exc['text_labels']
        df_inh["labels"] = rec.mlr_labels_inh['text_labels']
        df_exc[rec.filename] = rec.mlr_labels_exc['n_neurons_per_label']
        df_inh[rec.filename] = rec.mlr_labels_inh['n_neurons_per_label']
        tot_exc.append(len(rec.zscore_exc))
        tot_inh.append(len(rec.zscore_inh))
        sum_neurons_exc = np.sum(rec.mlr_labels_exc['n_neurons_per_label'])
        sum_neurons_inh = np.sum(rec.mlr_labels_inh['n_neurons_per_label'])
        percentage_exc =  ((sum_neurons_exc / len(rec.zscore_exc))*100)
        percentage_inh =  ((sum_neurons_inh / len(rec.zscore_inh)) * 100)
        perc_exc.append(percentage_exc)
        perc_inh.append(percentage_inh)
df_exc.loc[len(df_exc.index)] = tot_exc
df_exc.loc[len(df_exc.index)] = tot_inh
df_exc.loc[len(df_exc.index)] = perc_exc
df_exc.loc[len(df_exc.index)] = genotypes
df_exc.to_excel("output_exc.xlsx", sheet_name='mlr_model')

df_inh.loc[len(df_inh.index)] = tot_exc
df_inh.loc[len(df_inh.index)] = tot_inh
df_inh.loc[len(df_inh.index)] = perc_inh
df_inh.loc[len(df_inh.index)] = genotypes
df_inh.to_excel("output_inh.xlsx", sheet_name='mlr_model')

In [ ]:
dictionnary_data_exc = pd.read_excel("output_exc.xlsx", sheet_name="mlr_model")
df_exc = pd.DataFrame(dictionnary_data_exc)
genotype_exc = df_exc.iloc[-1]

act_wt_exc = df_exc[df_exc["labels"].str.startswith(('(1', 'Genotype'))]
df_act_exc = pd.DataFrame(act_wt_exc)
wt_act_exc = df_act_exc.iloc[:-1][genotype_exc[genotype_exc == 'WT'].index]
sum_wt_act_exc = wt_act_exc.sum(axis=0)
per_wt_act_exc = (sum_wt_act_exc/df_exc.iloc[-4][genotype_exc == 'WT'])*100
print(df_exc.iloc[-4])
print(df_exc.iloc[-4][genotype_exc == 'WT'])
act_ko_exc = df_exc[df_exc["labels"].str.startswith(('(1', 'Genotype'))]
df_act_exc = pd.DataFrame(act_ko_exc)
ko_act_exc = df_act_exc.iloc[:-1][genotype_exc[genotype_exc == 'KO-Hypo'].index]
sum_KO_act_exc = ko_act_exc.sum(axis=0)
per_ko_act_exc = (sum_KO_act_exc/df_exc.iloc[-4][genotype_exc == 'KO-Hypo'])*100
ylabel1 = '% neurons act'

desact_wt_exc = df_exc[df_exc["labels"].str.startswith(('(-1', 'Genotype'))]
df_desact_exc = pd.DataFrame(desact_wt_exc)
wt_desact_exc = df_desact_exc.iloc[:-1][genotype_exc[genotype_exc == 'WT'].index]
sum_wt_desact_exc = wt_desact_exc.sum(axis=0)
per_wt_desact_exc = (sum_wt_desact_exc / df_exc.iloc[-4][genotype_exc == 'WT']) * 100
desact_ko_exc = df_exc[df_exc["labels"].str.startswith(('(-1', 'Genotype'))]
df_desact_exc = pd.DataFrame(desact_ko_exc)
ko_desact_exc = df_desact_exc.iloc[:-1][genotype_exc[genotype_exc =='KO-Hypo'].index]
sum_KO_desact_exc = ko_desact_exc.sum(axis=0)
per_ko_desact_exc = (sum_KO_desact_exc / df_exc.iloc[-4][genotype_exc == 'KO-Hypo']) * 100
ylabel2_exc = '% neurons desact'

fig, ax = plt.subplots(1, 2, figsize=(15, 10))

boxplot(ax[0],list(per_wt_act_exc), list(per_ko_act_exc), ylabel1)
boxplot(ax[1],list(per_wt_desact_exc), list(per_ko_desact_exc),ylabel2_exc, ylim=[0,80])
 

In [ ]:
dictionnary_data_inh = pd.read_excel("output_inh.xlsx", sheet_name="mlr_model")
df_inh = pd.DataFrame(dictionnary_data_inh)
genotype_inh = df_inh.iloc[-1]

act_wt_inh = df_inh[df_inh["labels"].str.startswith(('(1', 'Genotype'))]
df_act_inh = pd.DataFrame(act_wt_inh)
wt_act_inh = df_act_inh.iloc[:-1][genotype_exc[genotype_inh == 'WT'].index]
sum_wt_act_inh = wt_act_inh.sum(axis=0)
per_wt_act_inh =  ((sum_wt_act_inh / df_inh.iloc[-3][genotype_inh == 'WT']) * 100)
act_ko_inh = df_inh[df_inh["labels"].str.startswith(('(1', 'Genotype'))]
df_act = pd.DataFrame(act_ko_inh)
ko_act_inh = df_act.iloc[:-1][genotype_inh[genotype_inh == 'KO-Hypo'].index]
sum_KO_act_inh = ko_act_inh.sum(axis=0)
per_ko_act_inh =  ((sum_KO_act_inh / df_inh.iloc[-3][genotype_inh == 'KO-Hypo']) * 100)
ylabel1_inh = '% neurons act'

desact_wt_inh = df_inh[df_inh["labels"].str.startswith(('(-1', 'Genotype'))]
df_desact_inh = pd.DataFrame(desact_wt_inh)
wt_desact_inh = df_desact_inh.iloc[:-1][genotype_inh[genotype_inh == 'WT'].index]
sum_wt_desact_inh = wt_desact_inh.sum(axis=0)
per_wt_desact_inh =  ((sum_wt_desact_inh / df_exc.iloc[-3][genotype_inh == 'WT']) * 100)
desact_ko_inh = df_inh[df_inh["labels"].str.startswith(('(-1', 'Genotype'))]
df_desact_inh = pd.DataFrame(desact_ko_inh)
ko_desact_inh = df_desact_inh.iloc[:-1][genotype_inh[genotype_inh == 'KO-Hypo'].index]
sum_KO_desact_inh = ko_desact_inh.sum(axis=0)
per_ko_desact_inh =  ((sum_KO_desact_inh / df_exc.iloc[-3][genotype_exc == 'KO-Hypo']) * 100)
ylabel2_inh = '% neurons desact'

fig_inh, axs_inh = plt.subplots(1, 2, figsize=(10, 6), sharey=False)
boxplot(axs_inh[0], list(per_wt_act_inh), list(per_ko_act_inh), ylabel1_inh)
axs_inh[0].set_xlabel("Inhibitory neurons activated", fontsize=20)
axs_inh[0].xaxis.set_label_coords(0.5, -0.05)
boxplot(axs_inh[1],list(per_wt_desact_inh), list(per_ko_desact_inh), ylabel2_inh,ylim=[0,40])
axs_inh[1].set_xlabel("Inhibitoy neurons desactivated", fontsize=20)
axs_inh[1].xaxis.set_label_coords(0.5, -0.05)
axs_inh[1].set_ylabel(None)


In [ ]:
df_select = pd.read_excel("output_exc.xlsx", sheet_name="mlr_model")
df_unselect = df_select.drop(df_select.index[-4:,])
genotype_exc = df_select.iloc[-1]
data_select = ['(1, 1, 0)', '(-1, -1, 0)', '(1, -1, 0)', '(-1, 1, 0)']
data_select_wt = df_select[df_select['labels'].isin(data_select)][genotype_exc[genotype_exc == 'WT'].index]
sum_data_select_wt = data_select_wt.sum(axis=0)
per_sum_data_select_wt =  ((sum_data_select_wt / df_exc.iloc[-4][genotype_exc == 'WT']) * 100)
data_select_ko = df_select[df_select['labels'].isin(data_select)][genotype_exc[genotype_exc == 'KO-Hypo'].index]
sum_data_select_ko = data_select_ko.sum(axis=0)
per_sum_data_select_ko =  ((sum_data_select_ko / df_exc.iloc[-4][genotype_exc == 'KO-Hypo']) * 100)
data_unselect_wt = df_unselect[~df_unselect['labels'].isin(data_select)][genotype_exc[genotype_exc == 'WT'].index]
sum_data_unselect_wt = data_unselect_wt.sum(axis=0)
per_sum_data_unselect_wt =  ((sum_data_unselect_wt / df_exc.iloc[-4][genotype_exc == 'WT']) * 100)
data_unselect_ko = df_unselect[~df_unselect['labels'].isin(data_select)][genotype_exc[genotype_exc == 'KO-Hypo'].index]
sum_data_unselect_ko = data_unselect_ko.sum(axis=0)
per_sum_data_unselect_ko = round((sum_data_unselect_ko / df_exc.iloc[-4][genotype_exc == 'KO']) * 100)
fig_exc, axs_exc = plt.subplots(1, 2, figsize=(10, 6), sharey=False)
boxplot(axs_exc[0], np.array(per_sum_data_select_wt, "int"), np.array(per_sum_data_select_ko,"int"), "Neurons (%)", ylim=[0,80])
axs_exc[0].set_xlabel("Detected stimulus selective", fontsize=23)
axs_exc[0].xaxis.set_label_coords(0.5, -0.05)
boxplot(axs_exc[1], np.array(per_sum_data_unselect_wt, "int"), np.array(per_sum_data_unselect_ko, "int"),"Neurons (%)")
axs_exc[1].set_xlabel("Stimulus & Reward", fontsize = 23)
axs_exc[1].xaxis.set_label_coords(0.5, -0.05)
axs_exc[1].set_ylabel(None)


In [ ]:
df_select_inh = pd.read_excel("output_inh.xlsx", sheet_name="mlr_model")
df_unselect_inh = df_select_inh.drop(df_select_inh.index[-4:,])
genotype_inh = df_select_inh.iloc[-1]
data_select_inh = ['(1, 1, 0)', '(-1, -1, 0)', '(1, -1, 0)', '(-1, 1, 0)']
data_select_wt_inh = df_select_inh[df_select_inh['labels'].isin(data_select_inh)][genotype_inh[genotype_inh == 'WT'].index]
sum_data_select_wt_inh = data_select_wt_inh.sum(axis=0)
per_sum_data_select_wt_inh = round((sum_data_select_wt_inh / df_inh.iloc[-4][genotype_inh == 'WT']) * 100)
data_select_ko_inh = df_select_inh[df_select_inh['labels'].isin(data_select_inh)][genotype_inh[genotype_inh == 'KO'].index]
sum_data_select_ko_inh = data_select_ko_inh.sum(axis=0)
per_sum_data_select_ko_inh = round((sum_data_select_ko_inh / df_inh.iloc[-4][genotype_inh == 'KO']) * 100)
data_unselect_wt_inh = df_unselect_inh[~df_unselect_inh['labels'].isin(data_select_inh)][genotype_inh[genotype_inh == 'WT'].index]
sum_data_unselect_wt_inh = data_unselect_wt_inh.sum(axis=0)
per_sum_data_unselect_wt_inh = round((sum_data_unselect_wt_inh / df_inh.iloc[-4][genotype_inh == 'WT']) * 100)
data_unselect_ko_inh = df_unselect_inh[~df_unselect_inh['labels'].isin(data_select_inh)][genotype_inh[genotype_inh == 'KO'].index]
sum_data_unselect_ko_inh = data_unselect_ko_inh.sum(axis=0)
per_sum_data_unselect_ko_inh = round((sum_data_unselect_ko_inh / df_inh.iloc[-4][genotype_inh == 'KO']) * 100)
fig_select_inh, axs_select_inh = plt.subplots(1, 2, figsize=(10, 6), sharey=False)
boxplot(axs_select_inh[0], np.array(per_sum_data_select_wt_inh, "int"), np.array(per_sum_data_select_ko_inh,"int"), "Neurons (%)", ylim=[0,8])
axs_select_inh[0].set_xlabel("Detected stimulus selective", fontsize=23)
axs_select_inh[0].xaxis.set_label_coords(0.5, -0.05)
boxplot(axs_select_inh[1], np.array(per_sum_data_unselect_wt_inh, "int"), np.array(per_sum_data_unselect_ko_inh, "int"),"Neurons (%)")
axs_select_inh[1].set_xlabel("Stimulus & Reward", fontsize = 23)
axs_select_inh[1].xaxis.set_label_coords(0.5, -0.05)
axs_select_inh[1].set_ylabel(None)
